In [1]:
#import PyDAQmx as nidaq

In [1]:
from numpy import *
import time
#import matplotlib.pylab as plt

In [2]:
#t = nidaq.Task()
#t.CreateAIVoltageChan("Dev2/ai0", None, nidaq.DAQmx_Val_Diff, 0, 10, nidaq.DAQmx_Val_Volts, None)

In [3]:
#t.CfgSampClkTiming("", 1000, nidaq.DAQmx_Val_Rising, nidaq.DAQmx_Val_FiniteSamps, 5000)
#t.StartTask()

In [5]:
#data = np.zeros((5000,), dtype=np.float64)
#read = nidaq.int32()
#t.ReadAnalogF64(5000, 5, nidaq.DAQmx_Val_GroupByChannel,
#   data, len(data), nidaq.byref(read), None)

In [3]:
#https://scipy-cookbook.readthedocs.io/items/Data_Acquisition_with_NIDAQmx.html
#!python numbers=disable
"""
This is an interpretation of the example program
C:\Program Files\National Instruments\NI-DAQ\Examples\DAQmx ANSI C\Analog Out\Generate Voltage\Cont Gen Volt Wfm-Int Clk\ContGen-IntClk.c
This routine will play an arbitrary-length waveform file.
This module depends on:
numpy
Adapted by Martin Bures [ mbures { @ } zoll { . } com ]
"""
# import system libraries
import ctypes
import numpy
import threading
# load any DLLs
nidaq = ctypes.windll.nicaiu # load the DLL
##############################
# Setup some typedefs and constants
# to correspond with values in
# C:\Program Files\National Instruments\NI-DAQ\DAQmx ANSI C Dev\include\NIDAQmx.h
# the typedefs
int32 = ctypes.c_long
uInt32 = ctypes.c_ulong
uInt64 = ctypes.c_ulonglong
float64 = ctypes.c_double
TaskHandle = uInt32
# the constants
DAQmx_Val_Cfg_Default = int32(-1)
DAQmx_Val_Volts = 10348
DAQmx_Val_Rising = 10280
DAQmx_Val_FiniteSamps = 10178
DAQmx_Val_ContSamps = 10123
DAQmx_Val_GroupByChannel = 0

##############################
class WaveformThread( threading.Thread ):
    """
    This class performs the necessary initialization of the DAQ hardware and
    spawns a thread to handle playback of the signal.
    It takes as input arguments the waveform to play and the sample rate at which
    to play it.
    This will play an arbitrary-length waveform file.
    """
    def __init__( self, waveform, sampleRate, device_name):
        #self.device_name = "Dev2/ao0"
        self.running = True
        self.sampleRate = sampleRate
        #self.device_name = device_name
        self.periodLength = len( waveform )
        self.taskHandle = TaskHandle( 0 )
        self.data = numpy.zeros( ( self.periodLength, ), dtype=numpy.float64 )
        # convert waveform to a numpy array
        for i in range( self.periodLength ):
            self.data[ i ] = waveform[ i ]
        # setup the DAQ hardware
        self.CHK(nidaq.DAQmxCreateTask("",
                          ctypes.byref( self.taskHandle )))
        self.CHK(nidaq.DAQmxCreateAOVoltageChan( self.taskHandle,
                                   device_name,
                                   "",
                                   float64(-10.0),
                                   float64(10.0),
                                   DAQmx_Val_Volts,
                                   None))
        self.CHK(nidaq.DAQmxCfgSampClkTiming( self.taskHandle,
                                "",
                                float64(self.sampleRate),
                                DAQmx_Val_Rising,
                                DAQmx_Val_FiniteSamps,
                                uInt64(self.periodLength)));
        self.CHK(nidaq.DAQmxWriteAnalogF64( self.taskHandle,
                              int32(self.periodLength),
                              0,
                              float64(-1),
                              DAQmx_Val_GroupByChannel,
                              self.data.ctypes.data,
                              None,
                              None))
        threading.Thread.__init__( self )
    def CHK( self, err ):
        """a simple error checking routine"""
        if err < 0:
            buf_size = 100
            buf = ctypes.create_string_buffer('\000' * buf_size)
            nidaq.DAQmxGetErrorString(err,ctypes.byref(buf),buf_size)
            raise RuntimeError('nidaq call failed with error %d: %s'%(err,repr(buf.value)))
        if err > 0:
            buf_size = 100
            buf = ctypes.create_string_buffer('\000' * buf_size)
            nidaq.DAQmxGetErrorString(err,ctypes.byref(buf),buf_size)
            raise RuntimeError('nidaq generated warning %d: %s'%(err,repr(buf.value)))
    def run( self ):
        counter = 0
        self.CHK(nidaq.DAQmxStartTask( self.taskHandle ))
    def stop( self ):
        self.running = False
        nidaq.DAQmxStopTask( self.taskHandle )
        nidaq.DAQmxClearTask( self.taskHandle )

In [4]:
from numpy import *
import pylab as plt

In [11]:
f_sampling = 250
"Here I tried to generate funtions in two different channels."
if __name__ == '__main__':
    import time
    # generate a time signal 5 seconds long with 250Hz sample rate
    t = numpy.arange( 0, 5, 1.0/f_sampling )
    # generate sine wave
    x = sin( t )
    y = cos( t )
    #plt.plot(t,x)
    mythread = WaveformThread( x, f_sampling, "Dev2/ao0")
   
    # start playing waveform
    mythread.start()
   
    # wait 5 seconds then stop
    time.sleep( 5 )
    mythread.stop()
    #mythread2 = WaveformThread( y, 250, "Dev2/ao1" )
    #mythread2.start()
    #time.sleep( 5 )
    
    #mythread2.stop()

In [16]:
'''Basic funtions to play with shutter and move Piezo controller in X and Y directions.'''
def shutter_on():
    t_shutter = numpy.arange( 0, 2, 1 )
    y = 10*t_shutter
    mythread = WaveformThread( y, 250, "Dev2/ao2")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()
def shutter_off():
    t_shutter = numpy.arange( 0, 2, 1 )
    y = 0*t_shutter
    mythread = WaveformThread( y, 250, "Dev2/ao2")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()


x_max = 16.6  # Piezo maximum travel in x direction (micron)
v_max = 10.0    # Voltale at DAQmx output to drive maximum travel in x direction
m_ = v_max/x_max # Required output voltage for 1 micron move
f_sampling = 250
def moveX(d_0, d_,v_): 
    t0_ = abs((d_-d_0)/v_)  # Calculate time for assigned length and speed
    t_ = numpy.arange( 0, t0_, 1.0/f_sampling )
    v = m_*d_0 + ((m_*d_-m_*d_0)/t0_)*t_  # Output voltage of DAQmx    
    mythread = WaveformThread( v, f_sampling, "Dev2/ao0")
    mythread.start()
    # wait t0_ seconds then stop
    time.sleep( t0_)
    mythread.stop()

def moveY(d_0, d_,v_): 
    t0_ = abs((d_-d_0)/v_)  # Calculate time for assigned length and speed
    t_ = numpy.arange( 0, t0_, 1.0/f_sampling )
    v = m_*d_0 + ((m_*d_-m_*d_0)/t0_)*t_  # Output voltage of DAQmx    
    mythread = WaveformThread( v, f_sampling, "Dev2/ao1")
    mythread.start()
    # wait t0_ seconds then stop
    time.sleep( t0_)
    mythread.stop()

def home_x():
    t = numpy.arange( 0, 2, 1 )
    y = 0*t
    mythread = WaveformThread( y, 250, "Dev2/ao0")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()
def home_y():
    t = numpy.arange( 0, 2, 1 )
    y = 0*t
    mythread = WaveformThread( y, 250, "Dev2/ao1")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()
def jump(x,y):
    t = numpy.arange( 0, 2, 1 )
    vx = m_*x*t
    mythread = WaveformThread( vx, 250, "Dev2/ao0")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()
    vy = m_*y*t
    mythread = WaveformThread( vy, 250, "Dev2/ao1")
    mythread.start()
    time.sleep( 1 )
    mythread.stop()

In [23]:
moveX(0,10,5)
#moveX(15,0,5)

In [24]:
jump(0,0)

In [25]:
'''Draw single line in an intact region'''
def single_line(v_): # v_ is the scanning speed
    home_y()
    home_x()
    shutter_on()
    moveY(0,20,v_)
    shutter_off()
    
'''Square'''    
def square(v,d,x,y):
    jump(x,y)
    #shutter_on()
    moveY(y,y+d,v)
    moveX(x,x+d,v)
    moveY(y+d,y,v)
    moveX(x+d,x,v)
    #shutter_off()
'''Multiple lines'''
def multi_line(v, d = 2.0):
    home_y()
    home_x()
    shutter_on()
    x = 0.0
    q = int((20/d)/2)
    for i in range(q):    
        moveY(0.0,20.0,v)
        #shutter_off()
        moveX(x,x+d,20)
        x = x + d
        #shutter_on()
        moveY(20.0,0.0,v) 
        moveX(x,x+d,20)
        x = x + d
        moveY(0.0,20.0,v)
    shutter_off()
    home_y()
    home_x()

In [637]:
home_y()
home_x()

In [636]:
shutter_on()

In [626]:
shutter_off()

In [26]:
square(v=1,d=10,x=0.0,y=0.0)

In [635]:
jump(8,8)

In [627]:
multi_line(2,3)

In [19]:
square(v=5,d=20,x=0.0,y=0.0)
#square(v=5,d=15,x=2.5,y=2.5)
#square(v=5,d=10,x=5.0,y=5.0)
#square(v=5,d=5,x=7.5,y=7.5)

ValueError: invalid number of arguments

In [619]:
shutter_on()

In [586]:
shutter_off()

In [274]:
x_max = 20e-6 # Piezo maximum travel in x direction
v_max = 10    # Voltale at DAQmx output to drive maximum travel in x direction
x_travel = 2e-6 # Length of straight lines we want to draw
v_travel_x = (v_max/x_max)*x_travel # Required volatge to draw x_travel length lines

speedx = 10e-6 # meter per second, speed of line drawing

t0x = x_travel/speedx  # Time to draw x_travel with speed speedx
LineSpace = 2e-6      # distance between each line
y_travel = LineSpace
v_travel_y = (v_max/x_max)*y_travel
print v_travel_y
speedy = speedx #meter per second, speed of line drawing in y direction
t0y = y_travel/speedy # Time to draw y_travel with speed speedx
f_sampling = 250
print t0x

1.0
0.2
